# Unfold a Polygon to create a Translation Surface

In [ ]:
from ipyvue_flatsurf import FlatSurface
from flatsurf import translation_surfaces, polygons, similarity_surfaces

In [ ]:
t = polygons.triangle(1, 1, 3)
B = similarity_surfaces.billiard(t)
S = B.minimal_cover('translation')

## A Visulation for a Flow Decomposition

In [ ]:
directions = S.saddle_connections(1)
v0 = directions[4].direction()
v1 = directions[5].direction()

In [ ]:
from flatsurf import GL2ROrbitClosure
O = GL2ROrbitClosure(S)
D = O.decomposition(v0)

In [ ]:
V = FlatSurface(D)
V

## Use the Same Glueings for Another Flow Decomposition

In [ ]:
import ipywidgets

D = O.decomposition(v1)
W = FlatSurface(D)
ipywidgets.link((V,'inner'), (W, 'forced'))
W

# Visualizing two Flow Decompositions as Orthogonal Directions

In [ ]:
A = matrix([v0, v1])
G, M = A.gram_schmidt()
N = G.transpose() * A.transpose().inverse()
T = S.apply_matrix(N, in_place=False)
T = T.delaunay_triangulation()

In [ ]:
from flatsurf import GL2ROrbitClosure
O = GL2ROrbitClosure(T)
D = O.decomposition((G[0][0], G[0][1]))
V = FlatSurface(D)
V

In [ ]:
D = O.decomposition((G[1][0], G[1][1]))
W = FlatSurface(D)
ipywidgets.link((V,'inner'), (W, 'forced'))
W